In [1]:
# conda install selenium 
# download chromedriver: https://sites.google.com/a/chromium.org/chromedriver/downloads      

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import numpy as np
from bs4 import BeautifulSoup
import re

### GET ALL PAGE LINKS FOR DIFFERCT CATEGORIES


In [2]:
#chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
chromedriver = "/Users/hiranya/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#url for one house in king county
driver = webdriver.Chrome(chromedriver)
url='https://www.shoes.com/womens-sneakers/category_189'

driver.get(url)
time.sleep(1)
for i in range(1,11):
    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{i*0.1});")
    time.sleep(0.1)

In [85]:
images = driver.find_elements_by_xpath('//a[@class="pt_thumb_link "]/img')

In [65]:
images = driver.find_elements_by_xpath('//img[@width="100%"]')

In [77]:
len(images)

36

In [3]:
def get_img_tags(url):
    links=[]
    for i in driver.find_elements_by_xpath('//a[@class="pt_thumb_link "]/img'):
        links.append(i.get_attribute('src'))
    return links

In [4]:
def get_shoe_pg(url):
    shoe_l=[]
    for i in driver.find_elements_by_xpath('//a[@class="pt_link"]'):
        shoe_l.append(i.get_attribute('href'))
    return shoe_l

In [5]:
def get_shoe_price(url):
    shoe_p=[]
    for i in driver.find_elements_by_xpath('//span[@class="pt_price"]'):
        shoe_p.append(i.text)
    return shoe_p

In [6]:
def get_shoe_desc(url):
    shoe_des=[]
    for i in driver.find_elements_by_xpath('//span[@class="pt_title"]'):
        shoe_des.append(i.text)
    return shoe_des

In [13]:
#check none tags
c=0
ind = 0
for i in itags_flats: 
    if i == None:
        c=c+1
        print(ind)
    ind=ind+1
print(c)
len(itags_flats)

0


900

In [14]:
len_im=[len(itags_flats)]
count_l=[]

for i in [itags_flats]:
    count=0
    for j in i:
        if j==None:
            count+=1
    count_l.append(count)

In [16]:
li=[]
for i in range(1):
    li.append(len_im[i]-count_l[i])
li

[900]

In [171]:
### COLLECT DATA FOR 3 MORE CATEGORIES

### Collect flats 

In [8]:

def get_url_flats(page):
    if page == 1:
        return 'https://www.shoes.com/womens-flats/category_239'
    return f'https://www.shoes.com/womens-flats/category_239?page={page}'

In [11]:
page = 1
itags_flats=[]
shoe_descs_flats=[]
shoe_pages_flats=[]
prices_flats=[]
while page < 26:
    url = get_url_flats(page)
    try:
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        time.sleep(2)
        for i in range(1,11):
            driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{i*0.1});")
            time.sleep(0.2)
    except:
        # out of pages!
        break
    #list_of_tables.append(pd.read_html(driver.page_source)[5])
    itag_flats=get_img_tags(url)
    itags_flats.extend((itag_flats))
    shoe_desc_flats=get_shoe_desc(url)
    shoe_descs_flats.extend((shoe_desc_flats))
    price_flats=get_shoe_price(url)
    prices_flats.extend((price_flats))
    shoe_page_flats=get_shoe_pg(url)
    shoe_pages_flats.extend((shoe_page_flats))
    
    page = page + 1
    driver.close()

In [17]:
#labels for ll images
sn_fl=['flats']*li[0]
#b_la=['boots']*li[1]
#sa_la=['sandals']*li[2]

In [18]:
data_fl=pd.DataFrame({'image_url':itags_flats,'page_url':shoe_pages_flats,\
                  'description':shoe_descs_flats,'price': prices_flats})

In [19]:
def func(x):
    return float(x.split()[0].replace('$','').replace(',',''))

In [20]:
data_fl['price']=data_fl['price'].apply(func)

In [22]:
data_fl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 4 columns):
image_url      900 non-null object
page_url       900 non-null object
description    900 non-null object
price          900 non-null float64
dtypes: float64(1), object(3)
memory usage: 28.2+ KB


In [23]:
data_fl.dropna(inplace=True)
data_fl.reset_index(drop=True, inplace=True)

In [24]:
data_fl.head()

,image_url,page_url,description,price
0,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/naturalizer-emiline-loaf...,Naturalizer Emiline Loafer (Women's),59.95
1,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/clarks-ashland-bubble/69...,Clarks Ashland Bubble (Women's),57.95
2,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/sam-edelman-loraine-hors...,Sam Edelman Loraine Horsebit Loafer (Women's),84.95
3,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/rockport-cobb-hill-paule...,Rockport Cobb Hill Paulette (Women's),79.99
4,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/skechers-gowalk-lite-ski...,Skechers GOwalk Lite Skimmer (Women's),49.95


In [25]:
#save images to local directory
import urllib.request
import os
os.mkdir('shoe_images_fl')
for i in range(len(data_fl)):
    urllib.request.urlretrieve(data_fl.loc[i,'image_url'], "shoe_images_fl/"+str(data_fl.loc[i,'page_url']).replace('/','_')+\
                               '_'+str(i)+'.jpg')


In [26]:
data_fl['label']=sn_fl

In [27]:
#get image path for all images 
for i in range(len(data_fl)) :  
    data_fl.loc[i,'image_path']="shoe_images_fl/"+str(data_fl.loc[i,'page_url']).replace('/','_')+\
                                   '_'+str(i)+'.jpg'

In [28]:
data_fl.head()

,image_url,page_url,description,price,label,image_path
0,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/naturalizer-emiline-loaf...,Naturalizer Emiline Loafer (Women's),59.95,flats,shoe_images_fl/https:__www.shoes.com_naturaliz...
1,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/clarks-ashland-bubble/69...,Clarks Ashland Bubble (Women's),57.95,flats,shoe_images_fl/https:__www.shoes.com_clarks-as...
2,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/sam-edelman-loraine-hors...,Sam Edelman Loraine Horsebit Loafer (Women's),84.95,flats,shoe_images_fl/https:__www.shoes.com_sam-edelm...
3,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/rockport-cobb-hill-paule...,Rockport Cobb Hill Paulette (Women's),79.99,flats,shoe_images_fl/https:__www.shoes.com_rockport-...
4,https://cdn-us-ec.yottaa.net/550c587c2106b06b5...,https://www.shoes.com/skechers-gowalk-lite-ski...,Skechers GOwalk Lite Skimmer (Women's),49.95,flats,shoe_images_fl/https:__www.shoes.com_skechers-...


In [29]:
data_fl.to_csv('data_fl.csv',sep=',',header=True,index=False)

In [30]:
data_fl.loc[800,:]

image_url      https://cdn-us-ec.yottaa.net/550c587c2106b06b5...
page_url       https://www.shoes.com/softwalk-sicily-flat/859721
description                       SoftWalk Sicily Flat (Women's)
price                                                      99.95
label                                                      flats
image_path     shoe_images_fl/https:__www.shoes.com_softwalk-...
Name: 800, dtype: object